In [1]:
import sys
import importlib
sys.path.insert(0, '/cndd/fangming/CEMBA/snmcseq_dev')
sys.path.insert(0, '/cndd2/fangming/projects/scf_enhancers/scripts/scf_enhancer_paper')
from __init__ import *
import __init__jupyterlab
importlib.reload(__init__jupyterlab)
from __init__jupyterlab import *

import re 
import tqdm
import pickle
import collections
import itertools
import tables

from scipy import sparse
from scipy import stats
from scipy import optimize 
import scipy.cluster.hierarchy as sch
from statsmodels.stats.multitest import multipletests

import snmcseq_utils
importlib.reload(snmcseq_utils)

<module 'snmcseq_utils' from '/cndd/fangming/CEMBA/snmcseq_dev/snmcseq_utils.py'>

In [2]:
dirc = '/cndd2/Public_Datasets/Cusanovich_Trapnell_Shendure_Nature_2018'

# Filter master connnections
- by cluster 5,15,29
- by time 

In [3]:
with snmcseq_utils.cd(dirc):
    masterconn = pd.read_csv('master_cicero_conns.txt', sep='\t')
    
masterconn.head()

,connection,Peak1,Peak2,coaccess,peak1.isproximal,peak1.tss.gene_name,peak1.tss.gene_id,peak2.isproximal,peak2.tss.gene_name,peak2.tss.gene_id,conn_type,cluster,subset_cluster,subcluster
0,chr10_100050458_100050896;chr10_100050979_1000...,chr10_100050458_100050896,chr10_100050979_100052296,0.295549,Yes,1700017N19Rik,ENSMUSG00000056912,Yes,1700017N19Rik,ENSMUSG00000056912,proximal_proximal,1,2,1_2
1,chr10_100050458_100050896;chr10_100050979_1000...,chr10_100050458_100050896,chr10_100050979_100052296,0.295549,Yes,1700017N19Rik,ENSMUSG00000056912,Yes,4930430F08Rik,ENSMUSG00000046567,proximal_proximal,1,2,1_2
2,chr10_100050458_100050896;chr10_100050979_1000...,chr10_100050458_100050896,chr10_100050979_100052296,0.295549,Yes,4930430F08Rik,ENSMUSG00000046567,Yes,1700017N19Rik,ENSMUSG00000056912,proximal_proximal,1,2,1_2
3,chr10_100050458_100050896;chr10_100050979_1000...,chr10_100050458_100050896,chr10_100050979_100052296,0.295549,Yes,4930430F08Rik,ENSMUSG00000046567,Yes,4930430F08Rik,ENSMUSG00000046567,proximal_proximal,1,2,1_2
4,chr10_100050979_100052296;chr10_100050458_1000...,chr10_100050979_100052296,chr10_100050458_100050896,0.295549,Yes,1700017N19Rik,ENSMUSG00000056912,Yes,1700017N19Rik,ENSMUSG00000056912,proximal_proximal,1,2,1_2


In [4]:
masterconn.shape

(17357676, 14)

In [5]:
masterconn['conn_type'].unique() #.isin([5, 15, 29])

array(['proximal_proximal', 'distal_distal', 'distal_proximal'],
      dtype=object)

In [6]:
filters = (
#     (masterconn['cluster'].isin([5, 15, 29]))
#     & 
    (masterconn['conn_type']=='distal_proximal')
)
filtered_conn = masterconn[filters]
filtered_conn.shape

(6204738, 14)

In [7]:
filtered_conn['idx'] = np.arange(len(filtered_conn)) 
filtered_conn

<ipython-input-7-0f9646abd971>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_conn['idx'] = np.arange(len(filtered_conn))


,connection,Peak1,Peak2,coaccess,peak1.isproximal,peak1.tss.gene_name,peak1.tss.gene_id,peak2.isproximal,peak2.tss.gene_name,peak2.tss.gene_id,conn_type,cluster,subset_cluster,subcluster,idx
54,chr10_120943650_120945081;chr10_121176203_1211...,chr10_120943650_120945081,chr10_121176203_121177843,0.216728,NaN,NaN,NaN,Yes,BC048403,ENSMUSG00000053684,distal_proximal,1,2,1_2,0
55,chr10_121176203_121177843;chr10_120943650_1209...,chr10_121176203_121177843,chr10_120943650_120945081,0.216728,Yes,BC048403,ENSMUSG00000053684,NaN,NaN,NaN,distal_proximal,1,2,1_2,1
56,chr10_122349263_122350845;chr10_122512482_1225...,chr10_122349263_122350845,chr10_122512482_122514164,0.251711,NaN,NaN,NaN,Yes,Mon2,ENSMUSG00000034602,distal_proximal,1,2,1_2,2
57,chr10_122512482_122514164;chr10_122349263_1223...,chr10_122512482_122514164,chr10_122349263_122350845,0.251711,Yes,Mon2,ENSMUSG00000034602,NaN,NaN,NaN,distal_proximal,1,2,1_2,3
66,chr10_129197483_129198278;chr10_129198381_1291...,chr10_129197483_129198278,chr10_129198381_129199164,0.348975,Yes,Olfr807,ENSMUSG00000050478,NaN,NaN,NaN,distal_proximal,1,2,1_2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17357663,chrX_98727644_98728644;chrX_98527425_98529073,chrX_98727644_98728644,chrX_98527425_98529073,0.495979,Yes,Taf1,ENSMUSG00000031314,NaN,NaN,NaN,distal_proximal,30,4,30_4,6204733
17357665,chrX_98834652_98836840;chrX_98527425_98529073,chrX_98834652_98836840,chrX_98527425_98529073,0.342224,Yes,Ogt,ENSMUSG00000034160,NaN,NaN,NaN,distal_proximal,30,4,30_4,6204734
17357667,chrX_98947256_98947585;chrX_98623954_98625731,chrX_98947256_98947585,chrX_98623954_98625731,0.223246,NaN,NaN,NaN,Yes,Nono,ENSMUSG00000031311,distal_proximal,30,4,30_4,6204735
17357668,chrX_99176854_99177132;chrX_99383845_99385084,chrX_99176854_99177132,chrX_99383845_99385084,0.432554,NaN,NaN,NaN,Yes,Rps4x,ENSMUSG00000031320,distal_proximal,30,4,30_4,6204736


In [8]:
filtered_conn.to_csv(os.path.join(dirc, 'master_cicero_conns_all_distalToProximal.txt'), 
                     sep='\t',
                     na_rep='NA',
                    )

# Clean up filtered connections
- check and understand
    - some duplicated connections - same connections annotated with different genes
    - no reciprocal duplicates 
- NOT remove duplicates; keep connections only
- liftOver connections to mm10
- conform to the existing format

In [9]:
filtered_conn['conn_type'].unique()
filtered_conn['peak1.isproximal'].unique()

array([nan, 'Yes'], dtype=object)

In [10]:
filtered_conn.shape

(6204738, 15)

### Prep bed to liftOver 

In [11]:
def get_region(region):
    chrom, start, end = region.split('_')
    return chrom, start, end

In [12]:
peak1_bed = pd.DataFrame()
peak1_bed['chrom'] = filtered_conn['Peak1'].apply(lambda x: get_region(x)[0])
peak1_bed['start'] = filtered_conn['Peak1'].apply(lambda x: get_region(x)[1])
peak1_bed['end'] = filtered_conn['Peak1'].apply(lambda x: get_region(x)[2])
peak1_bed['idx'] = filtered_conn['idx']

print(peak1_bed.shape)
peak1_bed.to_csv(os.path.join(dirc, 'liftover_all_distaltoproximal', 'peak1.bed'),
                 sep='\t', header=False, index=False,
                )

peak2_bed = pd.DataFrame()
peak2_bed['chrom'] = filtered_conn['Peak2'].apply(lambda x: get_region(x)[0])
peak2_bed['start'] = filtered_conn['Peak2'].apply(lambda x: get_region(x)[1])
peak2_bed['end'] = filtered_conn['Peak2'].apply(lambda x: get_region(x)[2])
peak2_bed['idx'] = filtered_conn['idx']

print(peak2_bed.shape)
peak2_bed.to_csv(os.path.join(dirc, 'liftover_all_distaltoproximal', 'peak2.bed'),
                 sep='\t', header=False, index=False,
                )

(6204738, 4)
(6204738, 4)


### liftOver 

In [13]:
res1 = pd.read_csv(os.path.join(dirc, 'liftover_all_distaltoproximal', 'peak1_mm10.bed'), 
                   sep='\t', header=None)
res2 = pd.read_csv(os.path.join(dirc, 'liftover_all_distaltoproximal', 'peak2_mm10.bed'), 
                   sep='\t', header=None)
    
conn_mm10 = pd.merge(res1, res2, on=3).rename(columns={
    3: 'idx', 
    
    '0_x': 'peak1_chr_mm10',
    '1_x': 'peak1_start_mm10',
    '2_x': 'peak1_end_mm10',
    
    '0_y': 'peak2_chr_mm10',
    '1_y': 'peak2_start_mm10',
    '2_y': 'peak2_end_mm10',
})

print(res1.shape, res2.shape, conn_mm10.shape)
conn_mm10.head()

(6204564, 4) (6204564, 4) (6204390, 7)


,peak1_chr_mm10,peak1_start_mm10,peak1_end_mm10,idx,peak2_chr_mm10,peak2_start_mm10,peak2_end_mm10
0,chr10,121506594,121508025,0,chr10,121739147,121740787
1,chr10,121739147,121740787,1,chr10,121506594,121508025
2,chr10,122912207,122913789,2,chr10,123075426,123077108
3,chr10,123075426,123077108,3,chr10,122912207,122913789
4,chr10,129760427,129761222,4,chr10,129761325,129762108


In [14]:
conn_mm10 = pd.merge(conn_mm10, filtered_conn, on='idx')
print(conn_mm10.shape)
conn_mm10.head()

(6204390, 21)


,peak1_chr_mm10,peak1_start_mm10,peak1_end_mm10,idx,peak2_chr_mm10,peak2_start_mm10,peak2_end_mm10,connection,Peak1,Peak2,...,peak1.isproximal,peak1.tss.gene_name,peak1.tss.gene_id,peak2.isproximal,peak2.tss.gene_name,peak2.tss.gene_id,conn_type,cluster,subset_cluster,subcluster
0,chr10,121506594,121508025,0,chr10,121739147,121740787,chr10_120943650_120945081;chr10_121176203_1211...,chr10_120943650_120945081,chr10_121176203_121177843,...,NaN,NaN,NaN,Yes,BC048403,ENSMUSG00000053684,distal_proximal,1,2,1_2
1,chr10,121739147,121740787,1,chr10,121506594,121508025,chr10_121176203_121177843;chr10_120943650_1209...,chr10_121176203_121177843,chr10_120943650_120945081,...,Yes,BC048403,ENSMUSG00000053684,NaN,NaN,NaN,distal_proximal,1,2,1_2
2,chr10,122912207,122913789,2,chr10,123075426,123077108,chr10_122349263_122350845;chr10_122512482_1225...,chr10_122349263_122350845,chr10_122512482_122514164,...,NaN,NaN,NaN,Yes,Mon2,ENSMUSG00000034602,distal_proximal,1,2,1_2
3,chr10,123075426,123077108,3,chr10,122912207,122913789,chr10_122512482_122514164;chr10_122349263_1223...,chr10_122512482_122514164,chr10_122349263_122350845,...,Yes,Mon2,ENSMUSG00000034602,NaN,NaN,NaN,distal_proximal,1,2,1_2
4,chr10,129760427,129761222,4,chr10,129761325,129762108,chr10_129197483_129198278;chr10_129198381_1291...,chr10_129197483_129198278,chr10_129198381_129199164,...,Yes,Olfr807,ENSMUSG00000050478,NaN,NaN,NaN,distal_proximal,1,2,1_2


In [15]:
conn_mm10.columns

Index(['peak1_chr_mm10', 'peak1_start_mm10', 'peak1_end_mm10', 'idx',
       'peak2_chr_mm10', 'peak2_start_mm10', 'peak2_end_mm10', 'connection',
       'Peak1', 'Peak2', 'coaccess', 'peak1.isproximal', 'peak1.tss.gene_name',
       'peak1.tss.gene_id', 'peak2.isproximal', 'peak2.tss.gene_name',
       'peak2.tss.gene_id', 'conn_type', 'cluster', 'subset_cluster',
       'subcluster'],
      dtype='object')

In [16]:
conn_mm10.drop(['idx'], axis=1).to_csv(
    os.path.join(dirc, 'master_cicero_conns_all_distalToProximal_mm10.txt'), 
    sep='\t', header=True, index=False
)

### generate two formats
- unique CICERO regions.bed
- enhancer-gene links

In [17]:
enh_gene_links = pd.concat([
    conn_mm10[conn_mm10['peak2.isproximal'].isnull()][
        ['peak2_chr_mm10', 'peak2_start_mm10', 'peak2_end_mm10', 'peak1.tss.gene_id', 'peak1.tss.gene_name', 'coaccess']].rename(
        columns={'peak2_chr_mm10': 'chrom', 
                 'peak2_start_mm10': 'start', 
                 'peak2_end_mm10': 'end', 
                 'peak1.tss.gene_id': 'gene_id', 
                 'peak1.tss.gene_name': 'gene_name', 
                }
    ),
    conn_mm10[conn_mm10['peak1.isproximal'].isnull()][
        ['peak1_chr_mm10', 'peak1_start_mm10', 'peak1_end_mm10', 'peak2.tss.gene_id', 'peak2.tss.gene_name', 'coaccess']].rename(
        columns={'peak1_chr_mm10': 'chrom', 
                 'peak1_start_mm10': 'start', 
                 'peak1_end_mm10': 'end', 
                 'peak2.tss.gene_id': 'gene_id', 
                 'peak2.tss.gene_name': 'gene_name', 
                }
    ),
    ])

print(enh_gene_links.shape)
enh_gene_links.head()


(6204390, 6)


,chrom,start,end,gene_id,gene_name,coaccess
1,chr10,121506594,121508025,ENSMUSG00000053684,BC048403,0.216728
3,chr10,122912207,122913789,ENSMUSG00000034602,Mon2,0.251711
4,chr10,129761325,129762108,ENSMUSG00000050478,Olfr807,0.348975
6,chr10,18318164,18319689,ENSMUSG00000075308,Gm10827,0.220091
7,chr10,18318164,18319689,ENSMUSG00000059554,Ccdc28a,0.220091


In [18]:
# uniq enh regions
uniq_regions = enh_gene_links[['chrom', 'start', 'end']].drop_duplicates()
print(uniq_regions.shape)
uniq_regions.head()

(232751, 3)


,chrom,start,end
1,chr10,121506594,121508025
3,chr10,122912207,122913789
4,chr10,129761325,129762108
6,chr10,18318164,18319689
10,chr10,30810155,30811376


In [19]:
# save both
enh_gene_links.to_csv(os.path.join(dirc, 'master_cicero_conns_all_EnhToGene_mm10.txt'), 
                   sep='\t', 
                   header=True, index=False,
                  )

uniq_regions.to_csv(os.path.join(dirc, 'master_cicero_conns_all_uniq_enhs_mm10.bed'), 
                    sep='\t',
                    header=False, index=False,
                   )

### Overlap uniq regions with enhancer lists 